## 범주형 변수 가공

EDA 시에는 흔히 dataframe['col_name'].value_counts() 활용  
범주마다 빈도가 다르고 범주의 수 자체도 다름  
**각 경우마다 가공법이 달라진다!**

시계열 변수도 pandas에서는 object(string)로 인식하기 때문에 전처리가 필수

### 가공 방식  
* Ordinal Encoding  
  * 각 범주를 0부터 n-1의 정수로 변환 
  * 순서가 존재하는 경우(유치원 - 초등학교 등)는 Ordinal, 순서가 없는 경우(성별) Nominal  
  * label encoding은 원래 종속 변수에 적용하는 것이나, ordinal과 비슷하게 동작함 
  * Cons: 한 번 출현한 범주도 각기 다른 수로 encoding됨 e.g. director col in movies 
    * Sol: 이러한 범주들을 묶어서 그룹 형성 e.g. 기타 / kaggler 모듈 활용
* One Hot Encoding
  * Ordinal encoding은 Tree-based 알고리즘에는 괜찮으나 LR, NN, SVM에서는 사용 불가능
    * 입력값이 수치형 변수여야 함 - 이때 ordinal encoding의 결과가 어떠한 대소 관계나 수치상 관계를 의미하지는 않음
  * 각각의 범주를 이진 변수로 변환
* Target Encoding  
  * 각 범주를 해당 범주에 속하는 종속변수의 평균값으로 변환
  * 종속변수에 대한 정보를 갖고 학습을 진행하기 때문에 과적합에 취약 - CV 또는 Smoothing과 함께 사용  
    * Smoothing: 빈도가 낮은 범주, 특히 한 번 등장하는 범주는 종속변수의 값을 고스란히 갖게 됨  
    이를 막기 위해 출현 빈도를 고려하여 전체 sample의 평균에 대해 weighted average로 변환
  * Q. 종속 변수가 없는 test set은 어떻게? 
    * 적용 불가능! 
    * TargetEncoder를 사용할 때, train set에는 fit_transform - test set에는 transform을 적용
  * Q. 문제의 형태 - multiclass, regression 등 - 에 상관 없이 적용 가능한가? 
    * 주의가 필요함 - 단순 평균? 정수로 반올림?
* Frequency Encoding
  * 각 범주를 해당 범주의 빈도로 변환
  * Label Encoding에 Grouping을 적용한 것과 효과
* Hashing Trick  
  * 각 범주에 hash 함수를 적용한 값을 변환 
  * Fit 과정이 필요 없어 가볍고 빠름 - large scale에서, 범주의 수가 많을 때 유리함
  


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!pip install kaggler

In [9]:
import kaggler
from lightgbm import LGBMRegressor
from matplotlib import rcParams, pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.sparse import hstack
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from warnings import simplefilter

In [10]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
simplefilter('ignore')

In [ ]:
from google.colab import files
files.upload()

In [3]:
!unzip movies.zip

Archive:  movies.zip
  inflating: submission.csv          
  inflating: movies_train.csv        
  inflating: movies_test.csv         


In [11]:
!ls

movies_test.csv  movies_train.csv  movies.zip  sample_data  submission.csv


In [12]:
data_dir = Path('.')
trn_file = data_dir / 'movies_train.csv'
seed = 42
target_col = 'box_off_num'

In [13]:
df = pd.read_csv(trn_file, index_col=0)
print(df.shape)
df.head()

(600, 11)


,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
title,,,,,,,,,,,
개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501
은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083
나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866
불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 600 entries, 개들의 전쟁 to 베를린
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   distributor     600 non-null    object 
 1   genre           600 non-null    object 
 2   release_time    600 non-null    object 
 3   time            600 non-null    int64  
 4   screening_rat   600 non-null    object 
 5   director        600 non-null    object 
 6   dir_prev_bfnum  270 non-null    float64
 7   dir_prev_num    600 non-null    int64  
 8   num_staff       600 non-null    int64  
 9   num_actor       600 non-null    int64  
 10  box_off_num     600 non-null    int64  
dtypes: float64(1), int64(5), object(5)
memory usage: 56.2+ KB


In [15]:
df['release_time'] = pd.to_datetime(df['release_time'])
print(df['release_time'].dtype)

datetime64[ns]


In [16]:
df.fillna(0, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 600 entries, 개들의 전쟁 to 베를린
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   distributor     600 non-null    object        
 1   genre           600 non-null    object        
 2   release_time    600 non-null    datetime64[ns]
 3   time            600 non-null    int64         
 4   screening_rat   600 non-null    object        
 5   director        600 non-null    object        
 6   dir_prev_bfnum  600 non-null    float64       
 7   dir_prev_num    600 non-null    int64         
 8   num_staff       600 non-null    int64         
 9   num_actor       600 non-null    int64         
 10  box_off_num     600 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(5), object(4)
memory usage: 56.2+ KB


In [17]:
num_cols = [x for x in df.columns if df[x].dtype in [np.int64, np.float64] and x != target_col]
cat_cols = ['distributor', 'genre', 'screening_rat', 'director']
print(f'    numeric ({len(num_cols)}):\t{num_cols}')
print(f'categorical ({len(cat_cols)}):\t{cat_cols}')

    numeric (5):	['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']
categorical (4):	['distributor', 'genre', 'screening_rat', 'director']


In [18]:
print(cat_cols)

['distributor', 'genre', 'screening_rat', 'director']


In [19]:
pd.DataFrame(df['distributor'].value_counts())

,distributor
CJ 엔터테인먼트,54
롯데엔터테인먼트,52
(주)NEW,30
(주)마운틴픽쳐스,29
인디스토리,26
...,...
전주국제영화제,1
스폰지이엔티,1
스크린조이,1
스튜디오 느림보,1


In [20]:
pd.DataFrame(df['genre'].value_counts())

,genre
드라마,221
다큐멘터리,93
멜로/로맨스,78
코미디,53
공포,42
액션,28
느와르,27
애니메이션,21
미스터리,17
SF,13


In [21]:
pd.DataFrame(df['screening_rat'].value_counts())

,screening_rat
청소년 관람불가,204
15세 관람가,202
12세 관람가,102
전체 관람가,92


In [22]:
pd.DataFrame(df['director'].value_counts())

,director
홍상수,7
노진수,4
신재호,4
전규환,4
장률,4
...,...
김일종,1
이현철,1
방은진,1
박헌수,1


In [23]:
df[['dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']] = df[['dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']].apply(np.log1p)
df[num_cols].describe()

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor
count,600.000000,600.000000,600.000000,600.000000,600.000000
mean,100.863333,5.305796,0.462197,4.026352,1.446130
std,18.097528,6.254561,0.555570,1.789517,0.446256
min,45.000000,0.000000,0.000000,0.000000,0.000000
25%,89.000000,0.000000,0.000000,2.890372,1.098612
50%,100.000000,0.000000,0.000000,4.424829,1.386294
75%,114.000000,12.837611,1.098612,5.579730,1.609438
max,180.000000,16.684279,1.791759,6.768493,3.258097


In [24]:
df['year'] = df['release_time'].dt.year
df['month'] = df['release_time'].dt.month
df.head()

,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,year,month
title,,,,,,,,,,,,,
개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,0.000000,0.000000,4.521789,1.098612,23398,2012,11
내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,13.965312,1.098612,5.961005,1.386294,7072501,2015,11
은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,12.304905,1.609438,5.840642,1.609438,6959083,2013,6
나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,10.081425,1.098612,3.044522,1.945910,217866,2012,7
불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,0.693147,0.693147,5.529429,1.098612,483387,2010,11


In [25]:
num_cols += ['year', 'month']
print(num_cols)

['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor', 'year', 'month']


In [26]:
features = num_cols + cat_cols
print(features)

['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor', 'year', 'month', 'distributor', 'genre', 'screening_rat', 'director']


In [27]:
rmse = lambda y, p: np.sqrt(mean_squared_error(y, p))
rmsle = lambda y, p: np.sqrt(mean_squared_error(np.log1p(y), np.log1p(p)))

In [28]:
from sklearn.preprocessing import OrdinalEncoder
df_cat = df.copy()
oe = OrdinalEncoder()
df_cat[cat_cols] = oe.fit_transform(df[cat_cols])
df_cat[cat_cols].head()

,distributor,genre,screening_rat,director
title,,,,
개들의 전쟁,80.0,10.0,3.0,393.0
내부자들,21.0,2.0,3.0,252.0
은밀하게 위대하게,21.0,10.0,1.0,357.0
나는 공무원이다,6.0,11.0,2.0,17.0
불량남녀,96.0,11.0,1.0,218.0


In [29]:
trn, tst = train_test_split(df_cat, test_size=.2, random_state=seed)
clf = LGBMRegressor(random_state=seed)
clf.fit(trn[features], np.log1p(trn[target_col]))
p = np.expm1(clf.predict(tst[features]))
print(f' RMSE:\t{rmse(tst[target_col], p):12.2f}')
print(f'RMSLE:\t{rmsle(tst[target_col], p):12.2f}')

 RMSE:	  1412591.02
RMSLE:	        1.98


In [30]:
from kaggler.preprocessing import LabelEncoder
df_cat = df.copy()
le = LabelEncoder(min_obs=2)
df_cat[cat_cols] = le.fit_transform(df[cat_cols])
df_cat[cat_cols].head()

,distributor,genre,screening_rat,director
title,,,,
개들의 전쟁,2,5,0,0
내부자들,6,6,0,6
은밀하게 위대하게,6,5,1,0
나는 공무원이다,3,3,3,0
불량남녀,63,3,1,0


In [31]:
trn, tst = train_test_split(df_cat, test_size=.2, random_state=seed)
clf = LGBMRegressor(random_state=seed)
clf.fit(trn[features], np.log1p(trn[target_col]))
p = np.expm1(clf.predict(tst[features]))
print(f' RMSE:\t{rmse(tst[target_col], p):12.2f}')
print(f'RMSLE:\t{rmsle(tst[target_col], p):12.2f}')

 RMSE:	  1177611.72
RMSLE:	        1.99


In [32]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
X = hstack((df[num_cols],
            ohe.fit_transform(df[cat_cols])))
print(X.shape)

(600, 664)


In [33]:
X_trn, X_tst, y_trn, y_tst = train_test_split(X, df[target_col], test_size=.2, random_state=seed)
clf = LGBMRegressor(random_state=seed)
clf.fit(X_trn, np.log1p(y_trn))
p = np.expm1(clf.predict(X_tst))
print(f' RMSE:\t{rmse(tst[target_col], p):12.2f}')
print(f'RMSLE:\t{rmsle(tst[target_col], p):12.2f}')

 RMSE:	  1239343.52
RMSLE:	        1.99


In [40]:
from kaggler.preprocessing import OneHotEncoder
ohe = OneHotEncoder(min_obs=2)
ohe.fit(df[cat_cols])
X = hstack((df[num_cols],
            ohe.fit_transform(df[cat_cols])))
print(X.shape)

(600, 187)


In [41]:
X_trn, X_tst, y_trn, y_tst = train_test_split(X, df[target_col], test_size=.2, random_state=seed)
clf = LGBMRegressor(random_state=seed)
clf.fit(X_trn, np.log1p(y_trn))
p = np.expm1(clf.predict(X_tst))
print(f' RMSE:\t{rmse(tst[target_col], p):12.2f}')
print(f'RMSLE:\t{rmsle(tst[target_col], p):12.2f}')

 RMSE:	  1183708.39
RMSLE:	        1.96


In [42]:
from kaggler.preprocessing import TargetEncoder
trn, tst = train_test_split(df, test_size=.2, random_state=seed)
te = TargetEncoder(cv=None)
trn[cat_cols] = te.fit_transform(trn[cat_cols], trn[target_col])
tst[cat_cols] = te.transform(tst[cat_cols])
trn[cat_cols].head()

,distributor,genre,screening_rat,director
title,,,,
돼지의 왕,721746.077154,1.998008e+05,3.511182e+05,723283.709843
청춘그루브,723430.057925,6.093409e+05,1.289088e+06,723430.057925
행복한 울릉인,722860.857509,7.472685e+04,1.529000e+05,723430.082604
옥희의 영화,718805.251941,6.093409e+05,3.511182e+05,711379.735399
권법형사 : 차이나타운,90921.244145,2.212935e+06,3.511182e+05,723429.982531


In [43]:
clf = LGBMRegressor(random_state=seed)
clf.fit(trn[features], np.log1p(trn[target_col]))
p = np.expm1(clf.predict(tst[features]))
print(f' RMSE:\t{rmse(tst[target_col], p):12.2f}')
print(f'RMSLE:\t{rmsle(tst[target_col], p):12.2f}')

 RMSE:	  1271836.20
RMSLE:	        3.40


In [44]:
trn, tst = train_test_split(df, test_size=.2, random_state=seed)
te = TargetEncoder()
trn[cat_cols] = te.fit_transform(trn[cat_cols], trn[target_col])
tst[cat_cols] = te.transform(tst[cat_cols])
trn[cat_cols].head()

,distributor,genre,screening_rat,director
title,,,,
돼지의 왕,723277.447850,267982.096480,348028.101562,723432.730281
청춘그루브,742425.489583,742425.489583,742425.489583,742425.489583
행복한 울릉인,768901.203125,10425.029522,122162.823956,768901.203125
옥희의 영화,722877.805490,813347.509259,380361.663366,721858.927057
권법형사 : 차이나타운,722864.097814,727096.433397,115692.865474,597129.065104


In [45]:
clf = LGBMRegressor(random_state=seed)
clf.fit(trn[features], np.log1p(trn[target_col]))
p = np.expm1(clf.predict(tst[features]))
print(f' RMSE:\t{rmse(tst[target_col], p):12.2f}')
print(f'RMSLE:\t{rmsle(tst[target_col], p):12.2f}')

 RMSE:	  1529036.45
RMSLE:	        2.12


In [46]:
from kaggler.preprocessing import FrequencyEncoder
df_cat = df.copy()
fe = FrequencyEncoder()
df_cat[cat_cols] = fe.fit_transform(df[cat_cols])
df_cat[cat_cols].head()

,distributor,genre,screening_rat,director
title,,,,
개들의 전쟁,52,28,204,1
내부자들,26,27,204,4
은밀하게 위대하게,26,28,202,1
나는 공무원이다,30,53,92,1
불량남녀,2,53,202,1


In [47]:
trn, tst = train_test_split(df_cat, test_size=.2, random_state=seed)
clf = LGBMRegressor(random_state=seed)
clf.fit(trn[features], np.log1p(trn[target_col]))
p = np.expm1(clf.predict(tst[features]))
print(f' RMSE:\t{rmse(tst[target_col], p):12.2f}')
print(f'RMSLE:\t{rmsle(tst[target_col], p):12.2f}')

 RMSE:	  1258846.99
RMSLE:	        1.96


In [48]:
from sklearn.feature_extraction import FeatureHasher
fh = FeatureHasher(n_features=128, input_type='string')
X = hstack([df[num_cols]] + [fh.fit_transform(df[col]) for col in cat_cols])
print(X.shape)

(600, 519)


In [49]:
X_trn, X_tst, y_trn, y_tst = train_test_split(X, df[target_col], test_size=.2, random_state=seed)
clf = LGBMRegressor(random_state=seed)
clf.fit(X_trn, np.log1p(y_trn))
p = np.expm1(clf.predict(X_tst))
print(f' RMSE:\t{rmse(tst[target_col], p):12.2f}')
print(f'RMSLE:\t{rmsle(tst[target_col], p):12.2f}')

 RMSE:	  1261559.18
RMSLE:	        1.95
